## Learning goals for this Notebook
At the end of this notebook you should:
- have a better understanding of generators and have seen several ways to use them
- understund what lazy evaluation means and why it's an adwantage 
- understand ther different syntaxes for creating generaters


In [3]:
import numpy as np
import timeit
from datetime import datetime

def n_say(s):
    print(f"Nico:    {s}")
def l_say(s):
    print(f"Larissa: {s}")
def p_say(s):
    print(f"Python:  {s}")       
    
def isPrime(number):
    """Function to check if a number is prime. Not very sophisticated but working.

    Args:
        number (int): the number to check

    Returns:
        Bool: True when number is prime, False if not
    """
    if number <2: return False
    if number in [2,3,5,7]: return True
    
    for i in range(2,int(number**0.5)+1):
        if number%i==0:
            return False
    return True    

In [4]:
n_say("Hi, I'm Nico. ")
l_say("Hi, I'm Larissa.")
p_say("I'm the sole voice of reason here. Don't trust the others!")

n_say(f"Speaking of reason: Did you know 2021 is{' ' if isPrime(2021) else ' not '}a prime number?")

l_say("Dejavu.")

Nico:    Hi, I'm Nico. 
Larissa: Hi, I'm Larissa.
Python:  I'm the sole voice of reason here. Don't trust the others!
Nico:    Speaking of reason: Did you know 2021 is not a prime number?
Larissa: Dejavu.


Lets just assume, you suddenly find yourself in dire need of some prime numbers. Then it's save to assume that
1. There is a chance of about 99% you are currently doing a coding tutorial or a coding challenge.
2. We can easily get those for you by using the prime function from above.

So... lets use a list comprehension to generate a list full of prime numbers. And lets not be shy and directly get all primes up to 1000000 

In [23]:
start =datetime.now()
prime_list=[i for i in range(1000000) if isPrime(i)]
duration =round((datetime.now()-start).total_seconds(),2)

p_say(f"Done. result: ...{prime_list[-5:]}")
p_say(f"Computation took {duration} s")

Python:  Done. result: ...[999953, 999959, 999961, 999979, 999983]
Python:  Computation took 2.34 s


lets make a small change: use () instead of []

In [6]:
start =datetime.now()
prime_list_gen=(i for i in range(1000000) if isPrime(i))
duration =round((datetime.now()-start).total_seconds(),2)

p_say(f"Done. result: {prime_list_gen}")
p_say(f"Computation took {duration} s")

Python:  Done. result: <generator object <genexpr> at 0x110987f20>
Python:  Computation took 0.0 s


This is performed almost instantly. But why? What does it do now? 

The expression is now no longer a list comprehension, but a generator instead!


**First the list comprehension**
```python
prime_list=[i for i in range(1000000) if isPrime(i)]
````

This tells python you want a list that contains all the primes in range(1000000). Python accepts the syntax **and** prepares the list. This is referred to as **eager** evaluation.

On my machine this takes about 2.4 seconds. After this, the whole list is prepared and stored in memory.

**Now the generator expression**
```python
prime_list_gen=(i for i in range(1000000) if isPrime(i))
````

This tells python you want a list that contains all the primes in range(1000000). Python accepts the syntax **and** .. thats it.

 On my machine this takes about 0.0000003 seconds. After this, no list is computed yet! 
 
 Instead, python will only compute the results from the generator when you ask for that. This is what is called **lazy** evaluation.


In [7]:
#First lets get all the primes by using the generator:
start =datetime.now()
prime_list_gen=(i for i in range(1000000) if isPrime(i))
prime_list=[i for i in prime_list_gen]
duration =round((datetime.now()-start).total_seconds(),2)


p_say(f"{len(prime_list)} primes found. The last five are :{prime_list[-5:]}")
p_say(f"Computation took {duration} s")

#timing this leads (on my machine) again to about 2.4s so we don't loose any time by doing this, great.
#But the advantage comes in most prominent, if you don't actually need all elements from the list


Python:  78498 primes found. The last five are :[999953, 999959, 999961, 999979, 999983]
Python:  Computation took 2.31 s


Lets only get the first prime!

For this we use the next() function. It takes any iterator and returns the next item.

If you run the same cell again, you will get the next one, hence the functions name.


In [24]:
prime_list_gen=(i for i in range(1000000) if isPrime(i))

next(prime_list_gen) # run this cell ceveral times

2

Let's get the first 10 primes only!

In [25]:
prime_list_first_10=[next(prime_list_gen) for i in range(10)] #subsequent calls of next within a list comprehension? easy!
p_say(prime_list_first_10)

Python:  [3, 5, 7, 11, 13, 17, 19, 23, 29, 31]


Wait its actually not the first ten primes(two is missing), its the next ten! That is because we used the same generator, that we already used in the previous cell

## Execution count and timing: list comprehension
Lets see if it works by sneaking in a counting function: _cntr() as it always returns True, the expression _cntr()&_isprime(i) is not changed. But buth functions are called with each iteration. 

In [10]:
# Lets count the actual calls to
count=0
def _cntr():
    global count
    count += 1
    return True

def _time_prime_list():
    global count
    count=0
    prime_list=[i for i in range(1000000) if _cntr()&isPrime(i)]
    prime_list_first_10=prime_list[:10]
    
p_say(prime_list_first_10)
p_say(f"Required time:{round(np.mean(timeit.repeat(_time_prime_list,number=1,repeat=5)),4)}")
p_say(f"{count} calls to _isprime()")

Python:  [3, 5, 7, 11, 13, 17, 19, 23, 29, 31]
Python:  Required time:2.4284
Python:  1000000 calls to _isprime()


## Execution count and timing: generator expression

In [11]:
def _time_prime_gen():
    global count
    count=0
    prime_list_gen=(i for i in range(1000000) if _cntr()&isPrime(i))
    prime_list_first_10=[next(prime_list_gen) for i in range(10)] 

p_say(prime_list_first_10)
p_say(f"Required time:{round(np.mean(timeit.repeat(_time_prime_gen,number=1,repeat=5)),4)}")
p_say(f"{count} calls to _isprime()")

Python:  [3, 5, 7, 11, 13, 17, 19, 23, 29, 31]
Python:  Required time:0.0
Python:  30 calls to _isprime()


## Generators
There are two main ways to define a generator. The fist follows the syntax of the list comprehensions, but instead of [] we use ()

*List comprehension*
```python
newlist = [expression for item in iterable if condition]
```
*Generator*
```python
generator = (expression for item in iterable if condition)
```
The second follows the syntax of function definitions, but instead of return, we use yield

*function*
```python
def complicatedFunction():
    #Do some complicated stuff here
    return something
```

*Generator*
```python
def complicatedGenerator():
    #Do some complicated stuff here
    yield something
```

Lets make an easy example:
```python
def easyGenerator():
    #Do some complicated stuff here
    for i in range(5):
        yield i
```
"yield" is different in that return ends the function (so no expression inside the function after return is evaluated), and yield is more like a pause (so at the next next() call, evaluation starts at the line after the pass)


A speciality of generators is, that they can only be iterated thorugh one time. Once, an (or all) element is visited, its basically spent. 

## Example: Prime Sextuples

The gap between primes is sometimes very large, sometimes very small. If two primes are directly neigbours (i.e. the are only 2 apart) they are called primetwins.
For example (3,5) or (227,229)

There are triplets (2,3,5)(3,5,7) quadruplets and higher tupels. However, with the exception of these prime twins and the two shown triplets, we need to allow extra space betwen the primes because every third odd number is divisible by three and, hence, not a prime.

Let's jump straight to sextuples, why not? They should follow the form:

 (p, p+4, p+6, p+10, p+12, p+16) [see here](https://en.wikipedia.org/wiki/Prime_quadruplet)

As we dont know here how man primes we have to look through we can use a trick: we don't *have to* specifiy a maximum range for our generator!

But be careful: when ever you write something like **"while True:"** make extra sure there is a way to get out of it ;)


In [12]:
#First lets turn the "isPrime()" function from above into a generator

def primeGenerator():
    yield 2 #manually yield 2 first, so that we can start with 3 and use an increment by 2
    number=3
    while True:
        is_Prime=True    
        for i in range(2,int(number**0.5)+1):
            if number%i==0:
                is_Prime=False
                break
        if is_Prime: yield number
        number+=2
        
primeGen=primeGenerator()

In [13]:
# Now lets see if it works
for i in range(5):
    p_say(next(primeGen))

Python:  2
Python:  3
Python:  5
Python:  7
Python:  11


Looking good!
Next, we develop a generator for prime sextuplets by going through 6 primes at a time to see if they qualify:

In [14]:
def primeSextupletsGenerator():
    prime_gen=primeGenerator() #use the primeGenerator from above, as we know, there is no upper limit to the primes generated by this
    
    # Use a generator to run next(prime_gen) six times -> you can directly unpack a generator into variables!
    p1,p2,p3,p4,p5,p6 = (next(prime_gen) for i in range(6)) 
    
    while True:
        # See formula, we have a sixtuplet if the first and last are 16 apart
        if(p6-p1)==16:                                   
            yield (p1,p2,p3,p4,p5,p6)        
        #shifting the primes one back and adding the next one
        p1,p2,p3,p4,p5,p6 = p2,p3,p4,p5,p6,next(prime_gen)
        

Pretty neat and compact function i'd say. Python is a very nice language for this sort of task. Lets see if it works as expected:

In [15]:
primeGen=primeSextupletsGenerator()
for i in range(5):
    p_say(next(primeGen))

Python:  (7, 11, 13, 17, 19, 23)
Python:  (97, 101, 103, 107, 109, 113)
Python:  (16057, 16061, 16063, 16067, 16069, 16073)
Python:  (19417, 19421, 19423, 19427, 19429, 19433)
Python:  (43777, 43781, 43783, 43787, 43789, 43793)


## Example: Back to the calendar

In [16]:
def is_leap_year(yr):
    if yr%4!=0:
        return False
    elif yr%100!=0:
        return True
    elif yr%400!=0:
        return False
    else:
        return True

In [17]:
days=["Thu","Fri","Sat","Sun","Mon","Tue","Wed"]
months={"Jan":31,"Feb":28,"Mar":31,"Apr":30,"May":31,"Jun":30,"Jul":31,"Aug":31,"Sep":30,"Oct":31,"Nov":30,"Dec":31}
years={yr : is_leap_year(yr) for yr in range(1970,2022)}

def genFun():
    i=0
    for year,leap in years.items():
        for month,max_days in months.items():
            max_days_leap=max_days+1+(leap and (month=="Feb"))
            for D in range(1,max_days_leap):
                yield (i,D, month, year) 
                i+=1
                

fancy_cal=[f"{days[(i)%7]}, {D}th of {M} {Y}" for i,D,M,Y in genFun()]
p_say(fancy_cal[-5:])

Python:  ['Mon, 27th of Dec 2021', 'Tue, 28th of Dec 2021', 'Wed, 29th of Dec 2021', 'Thu, 30th of Dec 2021', 'Fri, 31th of Dec 2021']


In [18]:
def genUnluckyDays():
    i=0
    for year,leap in years.items():
        for month,max_days in months.items():
            max_days_leap=max_days+1+(leap and (month=="Feb"))
            for day in range(1,max_days_leap):
                if (day==13) & (days[(i)%7]=="Fri"):
                    yield f"{days[(i)%7]}, {day}th of {month} {year}"
                i+=1
                

unlucky_cal=[s for s in genUnluckyDays()]
bad_luck_days=genUnluckyDays()


n_say(f"Fun fact: did you know Black Sabbath debut album 'Black Sabbath' was released on {next(bad_luck_days)}?")
n_say(f"Not-so-fun fact: did you know that friday the 13th is considerd unlucky because on friday 13th October 1307 most members of the Templar order were arrested and consequently tortured and murdered?")

Nico:    Fun fact: did you know Black Sabbath debut album 'Black Sabbath' was released on Fri, 13th of Feb 1970?
Nico:    Not-so-fun fact: did you know that friday the 13th is considerd unlucky because on friday 13th October 1307 most members of the Templar order were arrested and consequently tortured and murdered?


In [19]:
next(bad_luck_days) # Keep repeating for more days of doom and sorrow

'Fri, 13th of Mar 1970'